# Clean & Analyze Social Media

## Introduction

Social media has become a ubiquitous part of modern life, with platforms such as Instagram, Twitter, and Facebook serving as essential communication channels. Social media data sets are vast and complex, making analysis a challenging task for businesses and researchers alike. In this project, we explore a simulated social media, for example Tweets, data set to understand trends in likes across different categories.

## Prerequisites

To follow along with this project, you should have a basic understanding of Python programming and data analysis concepts. In addition, you may want to use the following packages in your Python environment:

- pandas
- Matplotlib

These packages should already be installed in Coursera's Jupyter Notebook environment, however if you'd like to install additional packages that are not included in this environment or are working off platform you can install additional packages using `!pip install packagename` within a notebook cell such as:

- `!pip install pandas`
- `!pip install matplotlib`

## Project Scope

The objective of this project is to analyze tweets (or other social media data) and gain insights into user engagement. We will explore the data set using visualization techniques to understand the distribution of likes across different categories. Finally, we will analyze the data to draw conclusions about the most popular categories and the overall engagement on the platform.

## Step 1: Importing Required Libraries

As the name suggests, the first step is to import all the necessary libraries that will be used in the project. In this case, we need pandas, numpy, matplotlib, seaborn, and random libraries.

Pandas is a library used for data manipulation and analysis. Numpy is a library used for numerical computations. Matplotlib is a library used for data visualization. Seaborn is a library used for statistical data visualization. Random is a library used to generate random numbers.

In [1]:
import csv # Python library used for reading and writing tabular data in CSV format.
import pandas as pd # Python library used for working with data sets. It has functions for analyzing, cleaning, exploring, and manipulating data. 
import json # Python library for encoding and decoding custom objects by using JSON encoder and decoder classes.
import numpy as np # Python library for adding support to large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
import seaborn as sns #Python library for statistical visualizations
import statistics as stat #Python library for statistical operations
import matplotlib.pyplot as plt # Python library for creating graphs
import random # Python library for generating random data

from scipy import stats # Python library for linear regression

In [24]:
# Define a List of Social Media Categories
categories = ["Food", "Travel", "Fashion", "Fitness", "Music", "Culture", "Family", "Health"]

# Create Random Data for the Project
random.seed(4) # Set the seed for generating the same random data.
data = {"Date":pd.date_range("2023-01-01", periods = 500),
       "Category":[random.choice(categories) for i in range(500)],
       "Likes":np.random.randint(0, 10000, size = 500)}

In [25]:
# Create a DataFrame from the data dictionary
df = pd.DataFrame(data)

In [26]:
# Review the Head of the DataFrame
df.head()

,Date,Category,Likes
0,2023-01-01,Fitness,4836
1,2023-01-02,Music,520
2,2023-01-03,Travel,2941
3,2023-01-04,Family,7048
4,2023-01-05,Health,5147


In [27]:
# Create a table showing the category counts
category_counts = df.Category.value_counts()
print(category_counts)

Category
Music      74
Fitness    65
Travel     62
Fashion    62
Food       61
Culture    61
Family     60
Health     55
Name: count, dtype: int64


In [10]:
# Set the seed once at the beginning
random.seed(42)

# Later in the script, random functions will still use the same seed
random_numbers1 = [random.randint(0, 100) for _ in range(5)]
random_numbers2 = [random.randint(0, 100) for _ in range(5)]

print(random_numbers1)  # Same output every time you run the notebook
print(random_numbers2) 

[81, 14, 3, 94, 35]
[31, 28, 17, 94, 13]


In [12]:
# Set the seed once at the beginning
random.seed(42)

# Later in the script, random functions will still use the same seed
random_numbers1 = [random.randint(0, 100) for _ in range(5)]
random_numbers2 = [random.randint(0, 100) for _ in range(5)]

print(random_numbers1)  # Same output every time you run the notebook
print(random_numbers2) 

[81, 14, 3, 94, 35]
[31, 28, 17, 94, 13]
